In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import datetime
import seaborn as sns
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import gc
import re
from collections import Counter
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [2]:
feature_train=pd.read_pickle("../processed_data/df_train2.pkl")
feature_test=pd.read_pickle("../processed_data/df_test2.pkl")

In [3]:
feature_train.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'label', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'truetime', 'day', 'hour', 'minute', 'mynull1', 'mynull2',
       'dev_ppi_pred', 'final_ppi', '160_height', '160_width', '160_ppi',
       'hw_ratio', 'hw_matrix', 'inch', 'vpn'],
      dtype='object')

In [4]:
feature_test.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'label', 'truetime', 'day', 'hour', 'minute', 'mynull1', 'mynull2',
       'dev_ppi_pred', 'final_ppi', '160_height', '160_width', '160_ppi',
       'hw_ratio', 'hw_matrix', 'inch', 'vpn'],
      dtype='object')

### 归一化normalize

In [5]:
for col in ['hw_ratio', 'inch','final_ppi', 'hw_matrix']:
    feature_train[col]=(feature_train[col] - feature_train[col].mean()) / (feature_train[col].std())
    feature_test[col]=(feature_test[col] - feature_test[col].mean()) / (feature_test[col].std())

## classfication

In [6]:
cate_feature = ['apptype', 'carrier', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'location', 'fea1_hash',"fea_hash", 'cus_type', '160_height', '160_width','160_ppi','vpn']
for item in cate_feature:
    le=preprocessing.LabelEncoder()
    feature_train[item] = le.fit_transform(feature_train[item])
    feature_test[item] = le.transform(feature_test[item])

prediction = feature_test[['apptype', 'carrier', 'dev_height','hour', 'minute',"timestamp","day",
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', '160_height', '160_width','160_ppi', 'hw_ratio', 'inch','final_ppi','vpn', 'hw_matrix']]
prediction_train = feature_train[['apptype', 'carrier', 'dev_height','hour', 'minute',"timestamp","day",
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', '160_height', '160_width','160_ppi', 'hw_ratio', 'inch','final_ppi','vpn', 'hw_matrix']]

prediction['label'] = 0
prediction_train['label'] = 0
prediction_train['label_prior'] =  feature_train.label.values


y_col = 'label'
x_col=['apptype', 'carrier', 'dev_height','hour', 'minute',"timestamp","day",
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', '160_height', '160_width','160_ppi', 'hw_ratio', 'inch','final_ppi','vpn', 'hw_matrix']
# x_col = ['apptype', 'carrier', 'dev_height', 'dev_ppi','timestamp',
#        'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
#        'Day', 'Hour', 'apptype_most_index', 'apptype_most_freq', 'carrier_most_index', 'carrier_most_freq',
#        'lan_most_index', 'lan_most_freq', 'media_id_most_index',
#        'media_id_most_freq', 'ntt_most_index', 'ntt_most_freq',
#        'os_most_index', 'os_most_freq', 'osv_most_index', 'osv_most_freq',
#        'package_most_index', 'package_most_freq', 'version_most_index',
#        'version_most_freq', 'location_most_index', 'location_most_freq',
#        'fea1_hash_most_index', 'fea1_hash_most_freq', 'cus_type_most_index',
#        'cus_type_most_freq', 'Day_most_index', 'Day_most_freq',
#        'Hour_most_index', 'Hour_most_freq']

In [7]:
model = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.4, importance_type='split',
               learning_rate=0.025, max_depth=7, metric=None,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10000, n_jobs=-1, num_leaves=128, objective=None,
               random_state=220, reg_alpha=0.3, reg_lambda=0.3, save_binary=True,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

oof = []
df_importance_list = []
n=10
# kfold = GroupKFold(n_splits=n)
kfold = KFold(n_splits=n,shuffle=True,random_state=220)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(feature_train[x_col], feature_train[y_col])):
    # if fold_id==6:
    #     continue
    X_train = feature_train.iloc[trn_idx][x_col]
    Y_train = feature_train.iloc[trn_idx][y_col]

    X_val = feature_train.iloc[val_idx][x_col]
    Y_val = feature_train.iloc[val_idx][y_col]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(
        X_train,
        Y_train,
        categorical_feature=cate_feature,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, Y_train), (X_val, Y_val)],
        verbose=100,
        eval_metric=["binary_error"],
        early_stopping_rounds=400
    )

    pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof = feature_train.iloc[val_idx].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict_proba(feature_test[x_col], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['label'] += pred_test / n
    pred_train= lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction_train.iloc[val_idx,-2] += pred_train

    df_importance = pd.DataFrame({
        'column': x_col,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)


Fold_1 Training ================================

Training until validation scores don't improve for 400 rounds
[100]	train's binary_error: 0.113358	valid's binary_error: 0.11566
[200]	train's binary_error: 0.10992	valid's binary_error: 0.11328
[300]	train's binary_error: 0.108104	valid's binary_error: 0.112
[400]	train's binary_error: 0.106449	valid's binary_error: 0.11136
[500]	train's binary_error: 0.105233	valid's binary_error: 0.1113
[600]	train's binary_error: 0.103804	valid's binary_error: 0.11052
[700]	train's binary_error: 0.102571	valid's binary_error: 0.1103
[800]	train's binary_error: 0.101509	valid's binary_error: 0.10998
[900]	train's binary_error: 0.100533	valid's binary_error: 0.1096
[1000]	train's binary_error: 0.0996489	valid's binary_error: 0.10984
[1100]	train's binary_error: 0.0987733	valid's binary_error: 0.10962
[1200]	train's binary_error: 0.09786	valid's binary_error: 0.1096
[1300]	train's binary_error: 0.0969822	valid's binary_error: 0.10962
[1400]	train's bi

In [8]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,timestamp,13458.666667
1,minute,11967.000000
2,location,10350.666667
3,cus_type,9786.555556
4,hw_ratio,9592.111111
5,inch,9014.777778
6,hour,8970.222222
7,hw_matrix,7974.555556
8,dev_height,7667.777778
9,fea1_hash,6796.222222


### 后接lr

In [10]:
# prediction['label'] = prediction['label']*10/9

In [111]:
X=prediction_train[['apptype', 'carrier', 'dev_height','hour', 'minute',"timestamp","day",'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', '160_height', '160_width','160_ppi', 'hw_ratio', 'inch','final_ppi','vpn', 'hw_matrix',"label"]].fillna(0)
y= prediction_train[["label_prior"]]
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=6)
# lr = SVC(max_iter=400,C=2,kernel='rbf',tol=0.001)
lr =LogisticRegression(C=1,)
lr = RandomForestClassifier(n_estimators=100,max_depth=25, random_state=10,n_jobs=-1,oob_score=True,min_samples_split=100,min_samples_leaf=20,max_features='sqrt')
# Fit on the train data
lr.fit(X_train, y_train)
# print (lr.oob_score_ )
score = lr.score(X_train, y_train)
print("The prediction score on the test data is {:.2f}%".format(score*100))
score = lr.score(X_test, y_test)
print("The prediction score on the test data is {:.2f}%".format(score*100))

The prediction score on the test data is 89.47%
The prediction score on the test data is 89.00%


In [90]:
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# X=prediction_train[['apptype', 'carrier', 'dev_height','hour', 'minute',"timestamp","day",'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', '160_height', '160_width','160_ppi', 'hw_ratio', 'inch','final_ppi','vpn', 'hw_matrix',"label"]].fillna(0)
# y= prediction_train[["label_prior"]]

# param_test1={'n_estimators': list(range(10,71,10))}
# gsearch1=GridSearchCV(
# estimator=RandomForestClassifier(min_samples_split=100,min_samples_leaf=20,max_depth=8,max_features='sqrt',random_state=10,n_jobs=-1),param_grid=param_test1,
# scoring='roc_auc',cv=5)
# gsearch1.fit(X,y)
# print(gsearch1.cv_results_,gsearch1.best_params_,gsearch1.best_score_)


{'mean_fit_time': array([1.87839212, 2.97730794, 4.10277395, 5.45916104, 7.11462851,
       8.21347399, 9.3156014 ]), 'std_fit_time': array([0.33236586, 0.09477538, 0.11157893, 0.08885174, 0.42922799,
       0.30449998, 0.13194417]), 'mean_score_time': array([0.15724292, 0.15833359, 0.20262074, 0.18448076, 0.18338523,
       0.30296011, 0.3106585 ]), 'std_score_time': array([0.01399997, 0.01106413, 0.02680967, 0.03408281, 0.05325976,
       0.03583993, 0.03366581]), 'param_n_estimators': masked_array(data=[10, 20, 30, 40, 50, 60, 70],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 10}, {'n_estimators': 20}, {'n_estimators': 30}, {'n_estimators': 40}, {'n_estimators': 50}, {'n_estimators': 60}, {'n_estimators': 70}], 'split0_test_score': array([0.93745058, 0.93841455, 0.9383683 , 0.93887455, 0.93846612,
       0.93837591, 0.93829484]), 'split1_test_score': array([0.93538718, 0.93649658, 0

In [112]:
# param_test2={"max_depth":range(10,31,3),"min_simples_split":range(50,201,20)}
# gsearch2=GridSearchCV(estimator=RandomForestClassifier(n_estimator=40,min_samples_leaf=20,max_features='sqrt',random_state=10,n_jobs=-1)),
# param_grid=param_test2,scoring="roc_auc",iid=False,cv=5)
# gsearch2.fit(X,y)  
# print(gsearch1.cv_results_,gsearch1.best_params_,gsearch1.best_score_)

In [18]:
# a.to_csv('submission_0611_0221.csv', index=False)

### 直接阈值

In [127]:
a = pd.DataFrame(prediction.index)
a['label'] =lgb_model.predict(feature_test[x_col], num_iteration=lgb_model.best_iteration_)

In [115]:
# threshold=0.5
# a["label"]=a.label.apply(lambda x:1 if x>threshold else 0)

### 后验

In [129]:
user_label=pd.DataFrame()
# user_label["uid"]=feature_train.groupby("android_id").count().index
# user_label=user_label.set_index("uid")
user_label["uid"]=feature_train.android_id.values
user_label["ntt"]=feature_train.ntt.values
temp=pd.DataFrame(feature_train.groupby(["android_id","ntt"]).label.mean())
temp=temp.reset_index()
temp.rename(columns={"android_id":"uid","label":"label_prior"},inplace=True)
# user_label["label_prior"]=feature_train.groupby(["android_id","ntt"]).label.transform("mean")

In [130]:
user_label=pd.merge(user_label,temp,on=["uid","ntt"],how="left")
user_label.drop_duplicates(inplace=True)

In [131]:
a["uid"]=feature_test.android_id.values
a["ntt"]=feature_test.ntt.values
a=pd.merge(a,user_label,how="left",on=["uid","ntt"])

In [132]:
def post(label,prior):
    n=len(label)
    count=0
    for i in range(n):
        if 0<=prior[i]<=0.1 and label[i]==1:
            label[i]=0
            count+=1
            # print(i)
        elif 0.9<=prior[i]<=1 and label[i]==0:
            label[i]=1
            count+=1
            # print(i)
        else:
            pass
    print(count)
    return label.values

In [133]:
a.label=post(a.label,a.label_prior)

367


In [134]:
a=a[["sid","label"]]
a.to_csv('submission_0617_2144.csv', index=False)

In [135]:
a

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1


In [32]:
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.4, importance_type='split',
               learning_rate=0.025, max_depth=7, metric=None,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10000, n_jobs=-1, num_leaves=128, objective=None,
               random_state=220, reg_alpha=3, reg_lambda=3, save_binary=True,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)